In [1]:
import math
import scipy.optimize
import time

In [2]:
numPeriods = 10 # total number of periods
marketSpotRates = [3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.55, 3.6, 3.65, 3.7]
driftParams = [5.]*len(marketSpotRates)
volatilityParam = 0.05
upMoveParam = 0.5
facevalue = 1 # 100 ????
maxIterations = 200



In [29]:


class Lattice:
    def printLattice(self):
        for t, level in enumerate(self.lattice):
            print('level {0}'.format(t))
            level = [ round(elem, 3) for elem in level ]
            print(', '.join(map(str, level)))



In [4]:
class CalibratedRateLattice(Lattice):
  def __init__(self, dP, vP, q):
    """
    dP --> a
    vP --> b
    """
    self.lattice = []
    self.lattice.append([dP[0]])
    for i in range(1, len(dP)):
      level = []
      for j in range(i+1):
        rate = dP[i]*math.exp(vP*j)
        level.append(rate)
      self.lattice.append(level)
    # print('Lattice is', self.lattice)


In [5]:

class EPLattice(Lattice):
    def __init__(self, F, rateLattice):
        self.lattice = []
        for i in range(len(rateLattice)+1):
            newLevel = []
            if i == 0:
                newLevel.append(F)
            else:
                level = rateLattice[i-1]
                for j in range(len(level)+1):
                    if j == 0:
                        discount = 1.+level[j]/100. # Interest rate 
                        price = .5*leftLevel[j]/discount
                    elif j == len(level):
                        discount = 1.+level[j-1]/100.
                        price = .5*leftLevel[j-1]/discount
                    else:
                        discount = 1.+level[j]/100.
                        price = .5*leftLevel[j]/discount
                        discount = 1.+level[j-1]/100.
                        price += .5*leftLevel[j-1]/discount
                    newLevel.append(price)

            leftLevel = newLevel
            # print('leftlevel is', i, leftLevel)
            self.lattice.append(newLevel)
    
    def getZCBPrices(self):
        return [sum(arr) for arr in self.lattice]
    
    def getSpotRates(self):
        zcbPrices= self.getZCBPrices()
        return [100.*((1./price)**(1./i)-1) for i, price in enumerate(zcbPrices) if i>0]

In [6]:
def G(x):
  rL = CalibratedRateLattice(x, volatilityParam, upMoveChance)
  epL = EPLattice(facevalue, rL.lattice[:])
  return [a-b for a, b in zip(epL.getSpotRates(), marketSpotRates)]

In [7]:
driftParams = [5]*14
volatilityParam = 0.005
upMoveChance = 0.5
rL = CalibratedRateLattice(driftParams, volatilityParam, upMoveChance)

facevalue = 1
print('spot rate lattice for 4 periods\n')
rL.lattice[0:4]
epL = EPLattice(facevalue, rL.lattice[:])
modelZCBPrices = epL.getZCBPrices()
modelSpotRates = epL.getSpotRates()
# print('epL is',  rL.lattice[0:3])
marketSpotRates = [7.3,7.62,8.1,8.45,9.2,9.64,10.12,10.45,10.75,11.22,11.55,
                                                            11.92,12.2,12.32]
print('spot rate squared error\n')
print(sum(i*i for i in G(driftParams)))

start_time = time.time()
maxIterations = 200
# Optimizing driftParams
driftParams = scipy.optimize.broyden1(G, driftParams, iter = maxIterations)
elapsedTime = time.time() - start_time

print(str(maxIterations) + " number of iterations used for otimization")
print(sum(i*i for i in G(driftParams)))

spot rate lattice for 4 periods

spot rate squared error

389.7698460708499
200 number of iterations used for otimization
5.842770387541903e-07


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/scipy/optimize/nonlin.py:1004: RuntimeWarning: invalid value encountered in true_divide
  d = v / vdot(df, v)


In [8]:


class OptionLattice(Lattice):
    def __init__(self, n, q, K, isCall, isAmerican, rateLattice, baseLattice):
        multiplier = 1 if isCall else -1
        clippedBase = baseLattice[:n+1]
        clippedRate = rateLattice[:n+1]
        self.lattice = []
        rightLevel = []
        print("Calculating options")
        for i, level in enumerate(reversed(clippedBase)):
            newLevel = []
            if i == 0:
                for j in range(len(level)):
                    newLevel.append(max(multiplier * (level[j]-K), 0))
            else:
                for j in range(len(level)):
                    earlyExercise = max(multiplier * (level[j]-K), 0)
                    discount = 1.+clippedRate[n-i][j]/100.
                    hold = (q*rightLevel[j+1] + (1-q)*rightLevel[j])/discount
                    if earlyExercise > hold and isAmerican:
                        print("At time {0}, it's better to early exercise {1} than hold {2}".format(n-i, earlyExercise, hold))
                    newPrice = max(hold, earlyExercise) if isAmerican else hold
                    newLevel.append(newPrice)
            rightLevel = newLevel
            self.lattice.insert(0, newLevel)



In [9]:
class SwapLattice(Lattice):
    def __init__(self, q, n, rf, firstPaymentTime, payFixed, rateLattice):
        clippedRate = rateLattice[:n+1]
        self.lattice = []
        rightLevel = []
        print("Calculating swaps")
        for i, level in enumerate(reversed(clippedRate)):
            newLevel = []
            if i == 0:
                for j in range(len(level)):
                    spotRate = clippedRate[n-i-1][j]/100.
                    payment = spotRate-rf if payFixed else rf-spotRate
                    newPrice = payment/(1+spotRate)
                    newLevel.append(newPrice)
            else:
                for j in range(len(level)):
                    spotRate = clippedRate[n-i-1][j]/100.
                    newPrice = (q*rightLevel[j+1]+(1-q)*rightLevel[j])/(1.+spotRate)
                    if n-i >= firstPaymentTime:
                        payment = spotRate-rf if payFixed else rf-spotRate
                        newPrice += payment/(1.+spotRate)
                    newLevel.append(newPrice)
            rightLevel = newLevel
            self.lattice.insert(0, newLevel)

In [10]:
# Q1
numPeriods = 10
marketSpotRates = [3.0,3.1,3.2,3.3,3.4,3.5,3.55,3.6,3.65,3.7]
driftParams = [5.]*len(marketSpotRates)
volatilityParam = .05
upMoveChance = .5
faceValue = 1.
maxIterations = 200

print('spot rate squared error')
print(sum(i*i for i in G(driftParams)))

start_time = time.time()
driftParams = scipy.optimize.broyden1(G, driftParams, iter=maxIterations)
elapsed_time = time.time() - start_time

print
print("%.2f ms elapsed over %d iterations" % (elapsed_time*1000, maxIterations))
print
print('spot rate squared error')
print(sum(i*i for i in G(driftParams)))

print
print('rate lattice')
rL = CalibratedRateLattice(driftParams, volatilityParam, upMoveChance)
# rL.printLattice()
print

numPeriods = 10
fixedRate = .039
firstPaymentTime = 4
paysFixed = True

sL = SwapLattice(upMoveChance, numPeriods, fixedRate, firstPaymentTime, paysFixed, rL.lattice[:])
print('swap lattice')
print
# sL.printLattice()
# print

numPeriods = 3
strikePrice = 0.
isCall = True
isAmerican = False

oL = OptionLattice(numPeriods, upMoveChance, strikePrice, isCall, isAmerican, 
                   rL.lattice[:], sL.lattice[:])
print('option lattice on swap')
print
# oL.printLattice()
# print
print('price w/ notional of $1M')
print(oL.lattice[0][0]*1000000)

spot rate squared error
36.145391259956114
183.14 ms elapsed over 200 iterations
spot rate squared error
2.091033601469336e-26
rate lattice
Calculating swaps
swap lattice
Calculating options
option lattice on swap
price w/ notional of $1M
4102.117637555908


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/scipy/optimize/nonlin.py:1004: RuntimeWarning: invalid value encountered in true_divide
  d = v / vdot(df, v)
/usr/local/lib/python3.6/dist-packages/scipy/optimize/nonlin.py:1004: RuntimeWarning: divide by zero encountered in true_divide
  d = v / vdot(df, v)


In [27]:
# Q2
numPeriods = 10
marketSpotRates = [3.0,3.1,3.2,3.3,3.4,3.5,3.55,3.6,3.65,3.7]
driftParams = [5.]*len(marketSpotRates)
volatilityParam = .1
upMoveChance = .5
faceValue = 1.
maxIterations = 200

print('spot rate squared error')
print(sum(i*i for i in G(driftParams)))

start_time = time.time()
driftParams = scipy.optimize.broyden1(G, driftParams, iter=maxIterations)
elapsed_time = time.time() - start_time

print
print("%.2f ms elapsed over %d iterations" % (elapsed_time*1000, maxIterations))
print
print('spot rate squared error')
print(sum(i*i for i in G(driftParams)))

print
print('rate lattice')
rL = CalibratedRateLattice(driftParams, volatilityParam, upMoveChance)
# rL.printLattice()
print

numPeriods = 10
fixedRate = .039
firstPaymentTime = 4
paysFixed = True

sL = SwapLattice(upMoveChance, numPeriods, fixedRate, firstPaymentTime, paysFixed, rL.lattice[:])
print('swap lattice')
print
# sL.printLattice()
# print

numPeriods = 3
strikePrice = 0.
isCall = True
isAmerican = False

oL = OptionLattice(numPeriods, upMoveChance, strikePrice, isCall, isAmerican, 
                   rL.lattice[:], sL.lattice[:])
print('option lattice on swap')
print
# oL.printLattice()
# print
print('price w/ notional of $1M')
print(oL.lattice[0][0]*1000000)

spot rate squared error
44324.72919417125
237.96 ms elapsed over 200 iterations
spot rate squared error
582344.6142807448
rate lattice
Calculating swaps
swap lattice
Calculating options
option lattice on swap
price w/ notional of $1M
2736.460732348142


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/scipy/optimize/nonlin.py:1004: RuntimeWarning: invalid value encountered in true_divide
  d = v / vdot(df, v)


In [39]:
class RateLattice(Lattice):    
    def __init__(self, n, S0, u, d):
        self.lattice = []
        for i in range(n+1):
            level = []
            for j in range(i+1):
                rate = S0 * u**j * d**(i - j)
                level.append(rate)
            self.lattice.append(level)

In [41]:
class HazardLattice(Lattice):    
    def __init__(self, n, a, b):
        self.lattice = []
        for i in range(n+1):
            level = []
            for j in range(i+1):
                rate = a*b**(j-.5*i)
                level.append(rate)
            self.lattice.append(level)

In [47]:
class  ZCBLattice(Lattice):
  def __init__(self, F, q, n, rateLattice, hazardLattice, 
               recoveryRate):
    self.lattice = []
    clippedRate = rateLattice[:n+1]
    clippedHazard = hazardLattice[:n+1]
    rightLevel = []
    for i, level in enumerate(reversed(clippedRate)):
      newLevel = []
      if i == 0:
        for j in range(len(level)):
          newLevel.append(F)
      else:
        for j in range(len(level)):
          # print('i and n-i are ', i, n-i)
          discount = 1 + clippedRate[n-i][j]/100
          hazard = clippedHazard[n-i][j]
          nondefaultValue = (q*rightLevel[j+1]+(1.-q)*rightLevel[j])*(1.-hazard)
          #nondefaultValue = (q*rightLevel[j+1] 
          #                   (1-q)*rightLevel[j])*(1-hazard)
          defaultValue = hazard*recoveryRate*F
          price = (nondefaultValue + defaultValue)/discount
          newLevel.append(price)
      rightLevel = newLevel
      self.lattice.insert(0, newLevel)


In [48]:
# Q3.
numPeriods = 10
startRate = 5
upMoveReturn = 1.1
downMoveReturn = 0.9
upMoveChance = 0.5

rL = RateLattice(numPeriods, startRate, upMoveReturn, downMoveReturn)
print('rate lattice\n')


driftParams = 0.01
volatilityParam = 1.01

hL = HazardLattice(numPeriods, driftParams, volatilityParam)
print('hazard lattice')

facevalue = 100
recoveryRate = 0.2
zL = ZCBLattice(facevalue, upMoveChance, numPeriods, rL.lattice[:],
                hL.lattice[:], recoveryRate)
print('zcb lattice')
zL.lattice



rate lattice

hazard lattice
zcb lattice


[[57.216858239429015],
 [63.033661378752626, 57.93139145235942],
 [68.59493947671312, 64.06743792132305, 59.002642014345355],
 [73.83792361896073, 69.93560612258966, 65.49626478332657, 60.51707364724876],
 [78.71996045857756,
  75.46359101739236,
  71.7052535935065,
  67.41597522913108,
  62.58655393372502],
 [83.21668031516685,
  80.6018453229398,
  77.54654814951435,
  74.00732972090235,
  69.95035300559505,
  65.35829574923974],
 [87.31937568522555,
  85.32166860098022,
  82.96297936487973,
  80.19589702297337,
  76.97512079888132,
  73.2618194916637,
  69.02946871507294],
 [91.0320424973309,
  89.61229862502474,
  87.92124964740066,
  85.91610470479456,
  83.55176793611062,
  80.7828949804018,
  77.56703196959934,
  73.8690237193042],
 [94.36842304668262,
  93.47755327573573,
  92.40866486484904,
  91.12986585049721,
  89.60550073954738,
  87.79663112441395,
  85.66204887810564,
  83.16000867956288,
  80.25086691522775],
 [97.3492758518275,
  96.93243836842603,
  96.42928459143658,

In [ ]:
# Q4 :
"""
Question 4
Quiz instructions

The true price of 5 different defaultable coupon paying bonds with non-zero recovery are specified in worksheet Calibration{\tt Calibration}Calibration in the workbook Assignment5_cds.xlsx.{\tt Assignment5\_cds.xlsx}.Assignment5_cds.xlsx. The interest rate is r=5%r = 5\%r=5% per annum. Calibrate the six month hazard rates A6{\tt A6}A6 to A16{\tt A16}A16 to by minimizing the Sum Error{\tt Sum \,Error}SumError ensuring that the term structure of hazard rates are non-decreasing. You can model the non-decreasing

hazard rates by adding constraints of the form A6≤A7,…,A15≤A16{\tt A6} \leq {\tt A7}, \ldots, {\tt A15} \leq {\tt A16}A6≤A7,…,A15≤A16. Report the hazard rate at time 000 as a percentage.

"""
# Ans. 1.87 

In [49]:
# Q5 :
"""
Modify the data on the CDS pricing{\tt CDS \,pricing}CDSpricing worksheet in the 
workbook bonds_and_cds.xlsx{\tt bonds\_and\_cds.xlsx}bonds_and_cds.xlsx to 
compute a par spread in basis points for a 5yr CDS with notional principal 
N=10N =10N=10 million assuming that
the expected recovery rate R=25%R = 25\%R=25%, the 3-month hazard rate is a flat 
1%1\%1%, and the interest rate is 5%5\%5% per annum.
"""
# Ans. 301.51

'\nModify the data on the CDS\u2009pricing{\tt CDS \\,pricing}CDSpricing worksheet in the \nworkbook bonds_and_cds.xlsx{\tt bonds\\_and\\_cds.xlsx}bonds_and_cds.xlsx to \ncompute a par spread in basis points for a 5yr CDS with notional principal \nN=10N =10N=10 million assuming that\nthe expected recovery rate R=25%R = 25\\%R=25%, the 3-month hazard rate is a flat \n1%1\\%1%, and the interest rate is 5%5\\%5% per annum.\n'